<h1>Case - Hurb Data Engineering (Pandas)</h1>
<p>Matheus Ramos Parracho</p>
<br></br>
<p>Apache Beam é uma framework que realmente nunca tinha utilizado antes. Eu tentei me adaptar para aprender e utilizar essa ferramenta, porém, dado ao meu tempo limitado essa semana por conta das várias provas de fim de período, realmente ficou bastante complicado para eu aprender a utilizar o Apache Beam.</p>
<br></br>
<p>Portanto, resolvi o Case com a ferramenta que tenho mais prática, que seria basicamente utilizando a lib "Pandas" para o gerenciamento dos arquivos csv.</p>
<br></br>
<p>Peço desculpas mais uma vez por não conseguir tempo para me adaptar ao Apache Beam, mas com certeza com um pouco de mais tempo e umas ajudinhas, eu conseguiria sim utilizar essa framework =)</p>

In [ ]:
#################################################################

<h3>Importando as bilbiotecas necessárias</h3>

In [186]:
import pandas as pd
import json

#pd.set_option("max_rows", None)
pd.set_option('max_columns', None)

<h3>Importando os arquivos</h3>

In [187]:
df_estados = pd.read_csv("EstadosIBGE.csv",delimiter=';')
df_covid = pd.read_csv("HIST_PAINEL_COVIDBR_28set2020.csv",delimiter=';')

In [188]:
df_estados.head()

,UF [-],Código [-],Gentílico [-],Governador [2019],Capital [2010],Área Territorial - km² [2019],População estimada - pessoas [2020],Densidade demográfica - hab/km² [2010],Matrículas no ensino fundamental - matrículas [2018],IDH <span>Índice de desenvolvimento humano</span> [2010],Receitas realizadas - R$ (×1000) [2017],Despesas empenhadas - R$ (×1000) [2017],Rendimento mensal domiciliar per capita - R$ [2019],Total de veículos - veículos [2018]
0,Acre,12,acriano,GLADSON DE LIMA CAMELI,Rio Branco,"164123,964",894470,"4,47",157646,"0,663","6632883,108","6084416,806",890,277831
1,Alagoas,27,alagoano,JOSE RENAN VASCONCELOS CALHEIROS FILHO,Maceió,"27843,295",3351543,"112,33",490587,"0,631","11950438,46","10460634,92",731,834827
2,Amapá,16,amapaense,ANTONIO WALDEZ GÓES DA SILVA,Macapá,"142470,762",861773,"4,69",136185,"0,708","5396417,145","4224464,088",880,195039
3,Amazonas,13,amazonense,WILSON MIRANDA LIMA,Manaus,"1559167,889",4207714,"2,23",705007,"0,674","17328459,43","15324896,56",842,883083
4,Bahia,29,baiano,RUI COSTA DOS SANTOS,Salvador,"564760,427",14930634,"24,82",2034711,"0,66","50191003,24",45570160,913,4139107


In [189]:
df_covid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,25/02/2020,9,210147125.0,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,26/02/2020,9,210147125.0,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,27/02/2020,9,210147125.0,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,28/02/2020,9,210147125.0,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,29/02/2020,9,210147125.0,2,1,0,0,NaN,NaN,NaN


<h3>Tratando os dados</h3>

Necessario: 
Agregado de informações por estado.
- Formato: CSV
- Informação: Regiao, Estado, UF, Governador, TotalCasos, TotalObitos

In [23]:
#como não encontrei uma tabela de "metadata" específico para esse dataset, fiz alguns testes para deduzir o
#significado de alguns valores.

In [ ]:
#a soma abaixo indica que as linhas em que a coluna "municipio" possui o valor "NaN"
#representam o caso geral para o estado indicado

In [ ]:
#para testar, utilizei o Amazonas como exemplo. Fiz a soma de todos os valores de casos acumulados de cada
#municipio no último dia da pesquisa e comparei com o valor do dos casos acumulados no último dia de pesquisa
#do "NaN" para o estado do Amazonas. Logo, confirmamos que o significado do "NaN" é representar
#o caso geral do estado, englobando todos os municipios.

In [42]:
sum(df_covid.loc[df_covid["estado"] == "AM"].loc[df_covid["data"] == "28/09/2020"]["casosAcumulado"].tolist()[1:])

136708

In [44]:
df_covid.loc[df_covid["estado"] == "AM"].loc[df_covid["data"] == "28/09/2020"]["casosAcumulado"].tolist()[0]

136708

In [ ]:
#na lista, a posicao "0", ou seja, a primeira linha, indica o numero dos casos na linha do "NaN"

In [ ]:
#Portanto, podemos concluir que esse valor significa o total de casos para o estado em questão.

In [ ]:
#separando as linhas com os valores totais e automatizando para todos os estados:

In [190]:
coduf = df_estados["Código [-]"].tolist()

In [191]:
df = pd.DataFrame()  
for index in coduf:
    df = df.append(df_covid.loc[df_covid["coduf"] == index].loc[df_covid["data"] == "28/09/2020"].iloc[0])
df.head()

,Recuperadosnovos,casosAcumulado,casosNovos,codRegiaoSaude,codmun,coduf,data,emAcompanhamentoNovos,estado,interior/metropolitana,municipio,nomeRegiaoSaude,obitosAcumulado,obitosNovos,populacaoTCU2019,regiao,semanaEpi
650,NaN,27885.0,11.0,NaN,NaN,12.0,28/09/2020,NaN,AC,NaN,NaN,NaN,657.0,0.0,881935.0,Norte,40.0
3254,NaN,86393.0,99.0,NaN,NaN,27.0,28/09/2020,NaN,AL,NaN,NaN,NaN,2062.0,5.0,3337357.0,Nordeste,40.0
1518,NaN,47935.0,140.0,NaN,NaN,16.0,28/09/2020,NaN,AP,NaN,NaN,NaN,704.0,2.0,845731.0,Norte,40.0
867,NaN,136708.0,292.0,NaN,NaN,13.0,28/09/2020,NaN,AM,NaN,NaN,NaN,4031.0,4.0,4144597.0,Norte,40.0
3688,NaN,306629.0,593.0,NaN,NaN,29.0,28/09/2020,NaN,BA,NaN,NaN,NaN,6648.0,49.0,14873064.0,Nordeste,40.0


In [ ]:
#criando o dataframe final

In [192]:
df_case = pd.DataFrame()
df_case["Regiao"] = df["regiao"]
df_case["UF"] = df["estado"]
df_case["TotalCasos"] = df["casosAcumulado"]
df_case["TotalObitos"] = df["obitosAcumulado"]

#para buscar os estados na ordem correta
estados = []
for index in coduf:
    estados += df_estados.loc[df_estados["Código [-]"] == index]["UF [-]"].tolist()
    
df_case["Estado"] = estados

#para buscar os governadores na ordem correta
governadores = []
for index in coduf:
    governadores += df_estados.loc[df_estados["Código [-]"] == index]["Governador [2019]"].tolist()
    
df_case["Governador"] = governadores    

df_case

,Regiao,UF,TotalCasos,TotalObitos,Estado,Governador
650,Norte,AC,27885.0,657.0,Acre,GLADSON DE LIMA CAMELI
3254,Nordeste,AL,86393.0,2062.0,Alagoas,JOSE RENAN VASCONCELOS CALHEIROS FILHO
1518,Norte,AP,47935.0,704.0,Amapá,ANTONIO WALDEZ GÓES DA SILVA
867,Norte,AM,136708.0,4031.0,Amazonas,WILSON MIRANDA LIMA
3688,Nordeste,BA,306629.0,6648.0,Bahia,RUI COSTA DOS SANTOS
2386,Nordeste,CE,239062.0,8921.0,Ceará,CAMILO SOBREIRA DE SANTANA
6075,Centro-Oeste,DF,190505.0,3203.0,Distrito Federal,IBANEIS ROCHA BARROS JUNIOR
4122,Sudeste,ES,129796.0,3524.0,Espírito Santo,JOSE RENATO CASAGRANDE
5858,Centro-Oeste,GO,202754.0,4529.0,Goiás,RONALDO RAMOS CAIADO
1952,Nordeste,MA,172176.0,3735.0,Maranhão,FLÁVIO DINO DE CASTRO E COSTA


<h3>Escrevendo os arquivos</h3>

In [193]:
#escrevendo o arquivo csv
df_case.to_csv('case_estados.csv',index=False)

In [194]:
#montando o arquivo json
lista_json = []
dicionario = {}
nome_colunas = df_case.columns.tolist()

for index in range(len(df_case.index)):
    for i in range(len(nome_colunas)):
        dicionario[nome_colunas[i]] = df_case.iloc[index].tolist()[i]
    lista_json.append(dicionario)
    dicionario = {}

In [214]:
json_case = json.dumps(lista_json, ensure_ascii=False).encode('latin1').decode('latin1')
print(json_case)

[{"Regiao": "Norte", "UF": "AC", "TotalCasos": 27885.0, "TotalObitos": 657.0, "Estado": "Acre", "Governador": "GLADSON DE LIMA CAMELI"}, {"Regiao": "Nordeste", "UF": "AL", "TotalCasos": 86393.0, "TotalObitos": 2062.0, "Estado": "Alagoas", "Governador": "JOSE RENAN VASCONCELOS CALHEIROS FILHO"}, {"Regiao": "Norte", "UF": "AP", "TotalCasos": 47935.0, "TotalObitos": 704.0, "Estado": "Amapá", "Governador": "ANTONIO WALDEZ GÓES DA SILVA"}, {"Regiao": "Norte", "UF": "AM", "TotalCasos": 136708.0, "TotalObitos": 4031.0, "Estado": "Amazonas", "Governador": "WILSON MIRANDA LIMA"}, {"Regiao": "Nordeste", "UF": "BA", "TotalCasos": 306629.0, "TotalObitos": 6648.0, "Estado": "Bahia", "Governador": "RUI COSTA DOS SANTOS"}, {"Regiao": "Nordeste", "UF": "CE", "TotalCasos": 239062.0, "TotalObitos": 8921.0, "Estado": "Ceará", "Governador": "CAMILO SOBREIRA DE SANTANA"}, {"Regiao": "Centro-Oeste", "UF": "DF", "TotalCasos": 190505.0, "TotalObitos": 3203.0, "Estado": "Distrito Federal", "Governador": "IBANE

In [212]:
#escrevendo o arquivo json
file = open('case_estados.json', 'w')
file.write(json_case)
file.close()